# 🎧 QualiVault: Prepare Audio
**Goal:** Convert raw audio files to 16-bit 16kHz FLAC, analyze channel separation, and count speakers.

1. Reads `processing_recipe.yaml`.
2. Converts/Merges audio files using `ffmpeg`.
3. Saves them to the configured `flac_output_folder` (outside the git repo).
4. **Analyzes Stereo Separation:** Checks if Left/Right channels are distinct.
5. **Counts Speakers:** Uses Pyannote AI to estimate speakers per channel.
6. Updates `processing_recipe.yaml` with the analysis results.

In [ ]:
%load_ext autoreload
%autoreload 2
import yaml
import warnings
from pathlib import Path

from qualivault import find_workspace_root

# ============================================
# PROJECT CONFIGURATION
# ============================================
PROJECT_NAME = 'YOUR_PROJECT_NAME'  # <-- Change this to work on different projects

workspace_root = find_workspace_root(Path.cwd())
project_root = workspace_root / 'projects' / PROJECT_NAME
config_path = project_root / 'config.yml'

if not project_root.exists():
    raise FileNotFoundError(f"❌ Project not found: {project_root}")
if not config_path.exists():
    raise FileNotFoundError(f"❌ Config not found: {config_path}")

# 1. Load Configuration

with open(config_path) as f:
    config = yaml.safe_load(f)

# Suppress warnings if configured
if config.get("suppress_warnings", True):
    warnings.filterwarnings("ignore")

# 2. Paths
recipe_path = project_root / "processing_recipe.yaml"
output_dir = (project_root / config['paths']['flac_output_folder']).resolve()
output_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Project root: {project_root}")
print(f"📄 Recipe:       {recipe_path}")
print(f"📂 FLAC output:  {output_dir}")

In [ ]:
# 3. Load Recipe
recipe_path = project_root / "processing_recipe.yaml"
if not recipe_path.exists():
    print("❌ Recipe not found! Please run '00_Setup_and_Scan.ipynb' first.")
    recipe = []
else:
    with open(recipe_path) as f:
        recipe = yaml.safe_load(f)
    print(f"✅ Loaded recipe with {len(recipe)} interviews.")

In [ ]:
# 4. Run Prepare + Analysis (resumable)
# Module handles conversion + analysis per interview and writes resumable progress to:
#   .qualivault/run_state.json
from qualivault.pipeline import prepare_audio_and_analyze

RESUME = True          # True: skip successful interviews, continue where it left off
MAX_INTERVIEWS = None  # e.g. 5 for testing, None for all

prepare_audio_and_analyze(
    project_root=project_root,
    config=config,
    recipe_path=recipe_path,
    resume=RESUME,
    max_items=MAX_INTERVIEWS,
)

print("\n✅ Done. You can stop/restart and resume later.")

In [ ]:
# 5. Notes
print("✅ Prepare/analysis is handled by the module.")
print("Progress is stored in: .qualivault/run_state.json")

In [ ]:
# 6. Save Updated Recipe
with open(recipe_path, 'w') as f:
    yaml.dump(recipe, f, sort_keys=False)
print(f"\n💾 Updated recipe saved to {recipe_path}")
print("   (You can now manually edit speaker counts in the YAML if needed)")

### 7. Quality Check
Loading all audio files at once can crash the kernel. Use the cell below to play specific files.

In [ ]:
# 7. Quality Check (optional)
# Play a specific FLAC by interview id.
import IPython.display as ipd

# Example: change to a specific id like "195"
target_id = "1"

output_dir = (project_root / config['paths']['flac_output_folder']).resolve()
flac_path = output_dir / f"Interview_{target_id}.flac"

if flac_path.exists():
    print(f"▶️ Playing: {flac_path.name}")
    display(ipd.Audio(filename=flac_path))
else:
    print(f"Not found: {flac_path}")